# Regular 3d fracture network

Extension to 3d of the test case 4.1 from the benchmark study [Flemisch 2017]. Both low permeable and high permeable fractures are considered. The domain is $(0, 1)^3$ and 9 fractures are present, see the `file geiger_3d.csv` for the coordinates. A graphical representation is given by<br>
<img src="geiger_network_0.png" width="50%"/>

In [1]:
import numpy as np
from porepy.fracs import importer

# geometrical tolerance
tol = 1e-8

# define the mesh size
h = 0.15
grid_kwargs = {}
grid_kwargs['mesh_size'] = {'mode': 'constant', 'value': h, 'bound_value': h, 'tol': tol}

# first line in the file is the domain boundary, the others the fractures
file_dfm = 'geiger_3d.csv'
gb, domain = importer.dfm_3d_from_csv(file_dfm, tol, **grid_kwargs)
gb.compute_geometry()

Use existing intersections
Use existing decomposition
Using old version of mesh size determination
Gmsh processed file successfully


Grid creation completed. Elapsed time 0.11263847351074219


Created 1 3-d grids with 3773 cells
Created 9 2-d grids with 792 cells
Created 69 1-d grids with 90 cells
Created 27 0-d grids with 27 cells




To following lines will be removed in a newer version.

In [2]:
%%capture
from porepy.grids.grid import FaceTag

internal_flag = FaceTag.FRACTURE
[g.remove_face_tag_if_tag(FaceTag.BOUNDARY, internal_flag) for g, _ in gb]

We introduce the class defining most of the data for the problem.

In [3]:
data_problem = {'domain': domain, 'tol': tol, 'aperture': 1e-4, 'km': 1, 'kf': 1e4}

Add the data to the grid bucket and the coupling permeability among objects of different dimensions.

In [4]:
from geiger_3d_data import DarcyModelData

gb.add_node_props(['is_tangential', 'problem', 'frac_num'])
for g, d in gb:
    d['problem'] = DarcyModelData(g, d, **data_problem)
    d['is_tangential'] = True

    if g.dim == 2:
        d['frac_num'] = g.frac_num*np.ones(g.num_cells)
    else:
        d['frac_num'] = -1*np.ones(g.num_cells)        

# Assign coupling permeability, the aperture is read from the lower dimensional grid
gb.add_edge_prop('kn')
for e, d in gb.edges_props():
    g_l = gb.sorted_nodes_of_edge(e)[0]
    aperture = gb.node_prop(g_l, 'param').get_aperture()
    d['kn'] = data_problem['kf'] / aperture

In [5]:
from porepy.numerics import elliptic

problem = elliptic.DualEllipticModel(gb)
problem.solve()
problem.split()

problem.pressure('pressure')
problem.discharge('discharge')
problem.project_discharge('P0u')
problem.save(['pressure', 'P0u', 'frac_num'])

To convert this script in pure python:

In [18]:
!jupyter nbconvert --to script geiger_3d.ipynb

[NbConvertApp] Converting notebook geiger_3d.ipynb to script
[NbConvertApp] Writing 599 bytes to geiger_3d.py
